In [5]:
import os 
from pathlib import Path
abspath = Path('C:\\Users\\harkum\\Documents\\6311\\Software')
os.chdir(abspath)

<h2 style=' color: 	#0bff00;border-bottom: 2px dotted #ff0000;'> Restore Register data to the Variable </h2>

```Restore__register address__variable name``` <i> example :~ Restore__varCB__0xCB "int offset idle" </i>

<h3 style='color:#ffac00'> Parse all Restore Register data from variable </h3>

In [6]:
from typing import List
import json
import random
import re 
import pandas as pd 
restore_instructions = {}
i=0
for sh in ['Digital','Reference','Boost','DAC+PA','AZ_COMP']:
    sheet = 'raw_data/'+sh+'_data.csv'
    data = pd.read_csv(sheet)
    for col in data.columns.to_list()[1:]:
        instructions = data[col][1].split('\n')
        for instruction in instructions:
            if re.match('restore__', instruction.lower()):
                restore_instructions.update({i : {
                    'page':sh,
                    'dft':col,
                    'ins':instruction
                }})
                i+=1
print(restore_instructions)
with open('raw_data/json/restore_parsing_raw_dump.json', 'w') as file:
    json.dump(restore_instructions, file)

{0: {'page': 'DAC+PA', 'dft': 'DAC-PA OFFSET CALIBRATION DIGITAL_IN (both PDM, TDM)\nThis procedure has the aim to measure the offset when the part will be in idle mode\n', 'ins': 'Restore__varCB__0xCB "int offset idle"'}, 1: {'page': 'DAC+PA', 'dft': 'DAC-PA OFFSET CALIBRATION DIGITAL_IN (both PDM, TDM)\nThis procedure has the aim to measure the offset when the part will be in idle mode\n', 'ins': 'Restore__varCC__0x+D5CC "int offset idle"'}, 2: {'page': 'DAC+PA', 'dft': 'DAC-PA OFFSET CALIBRATION DIGITAL_IN (both PDM, TDM)\nThis procedure has the aim to measure the offset when the part will be in idle mode\n', 'ins': 'Restore__varCE__0xCE "diff. ext-int in idle"'}, 3: {'page': 'DAC+PA', 'dft': 'DAC-PA OFFSET CALIBRATION DIGITAL_IN (both PDM, TDM)\nThis procedure has the aim to measure the offset when the part will be in idle mode\n', 'ins': 'Restore__varCF__0xCF "diff. ext-int in idle"'}}


<h3 style = 'color:#00f6ff'> Extract the Register Address from Restore Command</h3>

In [7]:
# This is the Approch 
pattern1=re.compile(r"\b(0[xX]+[0-9a-fA-F]+)+\[(.*?)\]")
re.findall(pattern1, '0xCB[7:4]')
'Restore__0xCB__varCB'.split('__')[1:]

['0xCB', 'varCB']

In [17]:
from typing import List

def extract_RestoreRegister__Instruction(instruction: str)->dict:
    # Remove the comments from the register
    if re.search(r"\"(.*?)\"",instruction):
        instruction = re.sub(r"\"(.*?)\"",'',instruction)
        
    pattern1=re.compile(r"\b(0[xX]+[0-9a-fA-F]+)+\[(.*?)\]")
    register ={}
    def register_format(reg:List):
        instruction_length = len(reg)
        register = {}
        # there is no bit field it is the default size
        if instruction_length == 2:
            if ":" in reg[1]:
                bits = reg[1].split(':')
                register={
                "RegAddr" : hex(int(reg[0],16)),
                "MSB" :hex(int(bits[1],16)),
                "LSB" : hex(int(bits[0],16)),
                "Data" : None,
                }
            else:
                register={
                    "RegAddr" : hex(int(reg[0],16)),
                    "MSB" : 7,
                    "LSB" : 0,
                    "Data" : hex(int(reg[1],16)),
                }
        # check for the bit field 
        elif instruction_length == 3:
            #check for the single bit or bit field 
            if ":" in reg[1]:
                bits = reg[1].split(':')
                register={
                "RegAddr" : hex(int(reg[0],16)),
                "MSB" :hex(int(bits[1],16)),
                "LSB" : hex(int(bits[0],16)),
                "Data" : hex(int(reg[2],16)),
                }
        else :
            #check for the single bit or bit field 
            register={
            "RegAddr" : hex(int(reg[0],16)),
            "MSB" :'0x7',
            "LSB" : '0x1',
            "Data" : None,
            }
        return register
    
    if 'restore__' in instruction.lower():
        if len(regs := [ re.findall(pattern1, i)[0] if re.match(pattern1, i) else i for i in instruction.split('__')[1:]  ] ) == 2:
            if restoreReg := register_format(regs[1]):
                register={
                        "restoreReg":restoreReg,
                        "var":regs[0],
                    }
        else:
            register =register
    else:
        register = register
    
    return register
extract_RestoreRegister__Instruction('Restore__varCB__0xCB "int offset idle"')

{'restoreReg': {'RegAddr': '0x0', 'MSB': '0x7', 'LSB': '0x1', 'Data': None},
 'var': 'varCB'}

<h3 style='color: #fff100'> Test restore Instruction </h3>

In [18]:
clean_restore_instructions = {}
for i,instruction_set in restore_instructions.items():
    # print(instruction_set)
    instruction = instruction_set.get('ins')
    if register_instruction_parse := extract_RestoreRegister__Instruction(instruction):
        # print(register_instruction_parse)
        clean_restore_instructions.update({
            i:{
                'set':instruction_set,
                'clean':register_instruction_parse
            }
        })
    else:
        print(instruction)
        clean_restore_instructions.update({
            i:None
        })
with open('raw_data/json/clean_restore_instructions.json', 'w') as file:
    json.dump(clean_restore_instructions, file)